In [ ]:
"""
!pip install -q sciunit
!pip install -q neo
!pip install -q elephant
!pip install -q gitpython
!pip install -q efel
"""

In [ ]:
#!pip install scipy seaborn
!pip install -U git+https://github.com/appukuttan-shailesh/hbp-validation-client.git
#!pip install --quiet fpdf PyPDF2 tabulate
#!pip install ./bbp-services-0.0.15/ ./task-types-0.0.10/ 
!pip install hbp_service_client

In [1]:
# First login in into the production version of the VF
from hbp_validation_framework import utils, ModelCatalog
x=ModelCatalog(username='pedroernesto')

import models
model_path = './str-fs_pv-20170919'
config_path = './morph_stats_config.yaml'
output_file = './models/model_predictions/NeuroM_MorphStats_prediction.json'

model2Test = models.NeuroM_MorphStats(model_path=model_path, config_path=config_path, output_file=output_file) 
# model2Test = models.CA1_laminar_distribution_synapses() 
utils.run_test(environment="dev", hbp_username="pedroernesto", model=model2Test, test_alias="morph_stats_Test", test_version="2.0")

/home/pedroernesto/miniconda2/lib/python2.7/site-packages/pandas/core/nanops.py:39: UserWarning: The installed version of bottleneck 0.8.0 is not supported in pandas and will be not be used
The minimum supported version is 1.0.0

  ver=ver, min_ver=_MIN_BOTTLENECK_VERSION), UserWarning)


----------------------------------------------
('Model name: ', 'NeuroM_MorphStats')
('Model type: ', <class 'models.NeuroM_MorphStats'>)
----------------------------------------------
----------------------------------------------
('Test name: ', "Cell's morpho-stats test")
('Test type: ', <class 'morphounit.tests.test_NeuroM_MorphStats.NeuroM_MorphStats_Test'>)
----------------------------------------------
----------------------------------------------
('Score: ', 1.7507281809075355)
Output files: 
./validation_results/neuroM_morph_softChecks/6af9b1033b730fd8347bab20a94e86666a5090fa*/20180412-165732/score_summary_OG060921A3-CH5-BC-H-ZK-60X-1-rep-cor.txt
./validation_results/neuroM_morph_softChecks/6af9b1033b730fd8347bab20a94e86666a5090fa*/20180412-165732/score_summary_C170897A-I1-rep-cor.txt
./validation_results/neuroM_morph_softChecks/6af9b1033b730fd8347bab20a94e86666a5090fa*/20180412-165732/score_summary_MTC070301C-IDA-rep-cor.txt
./validation_results/neuroM_morph_softChecks/6af9b

Result registered successfully!


u'4ac3a98e-8569-41c0-9e15-2e3ef52698e7'